# Comparison of DFT solvers

We compare four different approaches for solving the DFT minimisation problem,
namely a density-based SCF, a potential-based SCF, direct minimisation and Newton.

First we setup our problem

In [1]:
using DFTK
using LinearAlgebra

a = 10.26  # Silicon lattice constant in Bohr
lattice = a / 2 * [[0 1 1.];
                   [1 0 1.];
                   [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

model = model_LDA(lattice, atoms, positions)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=[3, 3, 3])

# Convergence we desire
tol = 1e-12
is_converged = DFTK.ScfConvergenceDensity(tol);

## Density-based self-consistent field

In [2]:
scfres_scf = self_consistent_field(basis; is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.846869607461                   -0.70    4.5
  2   -7.852312823491       -2.26       -1.53    1.0
  3   -7.852646225159       -3.48       -2.52    3.2
  4   -7.852646676519       -6.35       -3.37    2.5
  5   -7.852646685944       -8.03       -4.61    1.2
  6   -7.852646686729       -9.11       -5.63    3.2
  7   -7.852646686730      -11.94       -6.45    1.5
  8   -7.852646686730      -13.30       -7.75    2.5
  9   -7.852646686730      -14.75       -8.61    3.5
 10   -7.852646686730      -15.05       -9.03    1.2
 11   -7.852646686730   +  -15.05       -9.89    2.5
 12   -7.852646686730   +    -Inf       -9.73    1.0
 13   -7.852646686730   +  -14.75       -9.19    2.0
 14   -7.852646686730   +    -Inf       -8.31    2.0
 15   -7.852646686730   +    -Inf       -8.39    1.2
 16   -7.852646686730   +    -Inf       -8.23    1.8
 17   -7.852646686730      -14.57       -8.04 

## Potential-based SCF

In [3]:
scfres_scfv = DFTK.scf_potential_mixing(basis; is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   α      Diag
---   ---------------   ---------   ---------   ----   ----
  1   -7.846880474455                   -0.70           4.8
  2   -7.852557055434       -2.25       -1.63   0.80    2.0
  3   -7.852639996997       -4.08       -2.73   0.80    1.0
  4   -7.852646520044       -5.19       -3.39   0.80    2.0
  5   -7.852646678451       -6.80       -4.29   0.80    1.2
  6   -7.852646686619       -8.09       -4.82   0.80    2.0
  7   -7.852646686719      -10.00       -5.76   0.80    1.2
  8   -7.852646686730      -10.97       -6.36   0.80    2.5
  9   -7.852646686730      -12.93       -7.38   0.80    1.0
 10   -7.852646686730      -13.85       -7.92   0.80    2.5
 11   -7.852646686730      -14.75       -8.95   0.80    1.2
 12   -7.852646686730   +  -14.27       -9.94   0.80    2.5
 13   -7.852646686730   +    -Inf      -10.44   0.80    2.2
 14   -7.852646686730   +    -Inf      -11.68   0.80    1.8
 15   -7.852646686730      -14.45      -

## Direct minimization

In [4]:
scfres_dm = direct_minimization(basis; tol);

Iter     Function value   Gradient norm 
     0     1.395496e+01     3.641243e+00
 * time: 0.4709479808807373
     1     1.431951e+00     2.287508e+00
 * time: 0.7085630893707275
     2    -1.649548e+00     2.712881e+00
 * time: 0.73573899269104
     3    -3.625392e+00     2.143154e+00
 * time: 0.7741968631744385
     4    -5.164198e+00     1.696287e+00
 * time: 0.8122329711914062
     5    -6.700953e+00     1.047682e+00
 * time: 0.8530979156494141
     6    -6.789728e+00     1.221947e+00
 * time: 0.8808999061584473
     7    -7.421713e+00     7.283113e-01
 * time: 0.9086639881134033
     8    -7.534804e+00     6.095644e-01
 * time: 0.937222957611084
     9    -7.669522e+00     2.295460e-01
 * time: 0.9642598628997803
    10    -7.721928e+00     1.275654e-01
 * time: 0.9925520420074463
    11    -7.789073e+00     1.529589e-01
 * time: 1.0205140113830566
    12    -7.825534e+00     8.528827e-02
 * time: 1.0478270053863525
    13    -7.842618e+00     4.177171e-02
 * time: 1.0777580738067

## Newton algorithm

Start not too far from the solution to ensure convergence:
We run first a very crude SCF to get close and then switch to Newton.

In [5]:
scfres_start = self_consistent_field(basis; tol=1e-1);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.846856098393                   -0.70    4.8
  2   -7.852316473116       -2.26       -1.53    1.0


Remove the virtual orbitals (which Newton cannot treat yet)

In [6]:
ψ, _ = DFTK.select_occupied_orbitals(basis, scfres_start.ψ, scfres_start.occupation)
scfres_newton = newton(basis, ψ; tol);

n     Energy            log10(ΔE)   log10(Δρ)
---   ---------------   ---------   ---------
  1   -7.852646686706                   -2.54
  2   -7.852646686730      -10.62       -5.99
  3   -7.852646686730      -14.75      -12.48


## Comparison of results

In [7]:
println("|ρ_newton - ρ_scf|  = ", norm(scfres_newton.ρ - scfres_scf.ρ))
println("|ρ_newton - ρ_scfv| = ", norm(scfres_newton.ρ - scfres_scfv.ρ))
println("|ρ_newton - ρ_dm|   = ", norm(scfres_newton.ρ - scfres_dm.ρ))

|ρ_newton - ρ_scf|  = 6.965445807235536e-13
|ρ_newton - ρ_scfv| = 2.3024330893434884e-13
|ρ_newton - ρ_dm|   = 2.197986902514822e-10
